In [1]:
import gym
import tensorflow as tf
import numpy as np

In [2]:
env = gym.make('CartPole-v1')
actsize = env.action_space.n
stateSize = env.observation_space.shape[0]
sess = tf.Session()

In [3]:
class network(object):
    def __init__(self,session = None,input_dim = None,output_dim = None,lr = None):
        self.inputDim = input_dim
        self.outputDim = output_dim
        self.learningRate = lr
        #构建深度训练网络
        self.__buildNN()
        self.sess = session
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        
    def __buildNN(self):
        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)
        self.X = tf.placeholder(dtype=tf.float32,shape=(None,1,self.inputDim))
        self.Q_traget = tf.placeholder(dtype=tf.float32, shape=(None,1,self.outputDim), name='Q_target')
        h1 = tf.layers.dense(inputs=self.X, units=16, activation=tf.nn.relu,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        h2 = tf.layers.dense(inputs=h1, units=16, activation=tf.nn.relu,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        self.Y = tf.layers.dense(inputs=h1, units=self.outputDim,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        self.loss = tf.reduce_mean(tf.squared_difference(self.Q_traget,self.Y))  #TD-ERROR loss
        self._train_op = tf.train.RMSPropOptimizer(self.learningRate).minimize(self.loss)    

    def train(self,qvalue = None, predict = None):
        _, cost = self.sess.run([self._train_op,self.loss],feed_dict={self.X:predict, self.Q_traget:qvalue})
        return cost
        
    def predict(self,Input = None):
        evalData = self.sess.run(self.Y,feed_dict={self.X:Input})
        return evalData
        

In [4]:
nn = network(sess,input_dim = stateSize, output_dim = actsize,lr = 0.01)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
class dqn(object):
    def __init__(self,env = None,network= None,gamma = 0.9,epsilon = 0.5):
        self._nn = network
        self._gamma = gamma
        self._epsilon = epsilon
        self._env = env
    
    def preprocessInput(self,inputdata):
        data = np.array(inputdata).reshape(1,1,4)
        return data
    
    def _qtarget(self,s,a):
        next_state, reward, done, _ = self._env.step(a)
        next_state = self.preprocessInput(next_state)
        idx = 0
        if not done:
            value = self._nn.predict(next_state)
            idx = sess.run(tf.math.argmax(value[0],1))
            max_action = value[0][0][idx[0]]
            qtarget = reward + self._gamma * max_action
        else:
            qtarget = -10
        newvalue = self._nn.predict(s)
        newvalue[0][0][a] = qtarget
        return newvalue,next_state,done
        
    def learning(self,state,action):
        qvalue,next_state,done = self._qtarget(state,action)
        cost = self._nn.train(qvalue,state)
        return next_state,done
    
    def greedy_act(self,state):
        pY = self._nn.predict(state)
        idx = sess.run(tf.math.argmax(pY[0],1))
        return idx[0]
        

In [11]:
DQN = dqn(env,nn)
# pY = nn.predict(state)
# idx = sess.run(tf.math.argmax(pY[0],1))
for i in range(500):
    state = env.reset()
    state = np.array(state).reshape(1,1,4)
    action = DQN.greedy_act(state)
    count = 0
    for j in range(200):
        next_state,done = DQN.learning(state,action)
        if done:
            print("Failed:{} at {}/500".format(count,i))
            break
        state = next_state
        action = DQN.greedy_act(state)
        count += 1
    
    

Failed:10 at 0/500
Failed:9 at 1/500
Failed:9 at 2/500
Failed:10 at 3/500
Failed:10 at 4/500
Failed:14 at 5/500
Failed:46 at 6/500
Failed:35 at 7/500
Failed:17 at 8/500
Failed:10 at 9/500
Failed:8 at 10/500
Failed:9 at 11/500
Failed:10 at 12/500
Failed:9 at 13/500
Failed:9 at 14/500
Failed:9 at 15/500
Failed:8 at 16/500
Failed:14 at 17/500
Failed:13 at 18/500
Failed:14 at 19/500
Failed:10 at 20/500
Failed:16 at 21/500
Failed:14 at 22/500
Failed:44 at 23/500
Failed:9 at 24/500
Failed:9 at 25/500
Failed:7 at 26/500
Failed:9 at 27/500
Failed:10 at 28/500
Failed:8 at 29/500
Failed:9 at 30/500
Failed:8 at 31/500
Failed:9 at 32/500
Failed:8 at 33/500
Failed:8 at 34/500
Failed:8 at 35/500
Failed:9 at 36/500
Failed:9 at 37/500
Failed:8 at 38/500
Failed:10 at 39/500
Failed:9 at 40/500
Failed:8 at 41/500
Failed:8 at 42/500
Failed:18 at 43/500
Failed:19 at 44/500
Failed:42 at 45/500
Failed:19 at 46/500
Failed:18 at 47/500
Failed:21 at 48/500
Failed:27 at 49/500
Failed:30 at 50/500
Failed:17 at 51

KeyboardInterrupt: 